# AI VISION WORKSHOP
Our first step, as in most Python programs, is to import the external Python code modules that we will be using.  Note this first step assumes that your Python virtual environment is activated and all of these packages/modules have already been installed in the environment.

In [1]:
import os
import cv2
from PIL import Image
import webcolors
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential
from azure.storage.blob import BlobServiceClient
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
import serial
import time

Next we need to set the credentials that we'll use to connect to the Azure AI services and get responses to our API calls.  Note that these credentials are unique to each individual user.  It is best practice to store these in a separate local file and then import them into your code.  The local file is marked so as not to be uploaded or shared in any version control systems such as GitHub.

Be sure to log in to your Azure account (portal.azure.com), create a 'Computer Vision' resource and a storage blob (details in workshop lab instructions), and note the unique endpoint, security key, and storage blob connection string so that you can paste them into the code below...

In [ ]:
az_endpoint = "<insert your unique endpoint here>"
key = "<insert your unique key here"  
storage_connection_string = "<insert your unique storage connection string here>"

Now we'll set some static global variables that will be used later in the program...

In [7]:
# Static global variables are defined here...
img_file = '.\\WEBCAM-IMAGES\\sort-object.jpg'  # The name (and file path) of the image file which will be captured and analyzed
com_port = 'COM4'  # COM port for Arduino board connection (USB)

Next we'll create the basic functions of our code that depend on your local computer:

1. Capturing an image from your webcam
2. Communicating via the USB serial (COM) port to an attached Arduino microcontroller board in order to rotate a servo motor

We'll start with the webcam and define some functions to conect to it and capture a still frame...

In [3]:
def get_image_frame_from_webcam():
    """Returns a single frame (image) from the webcam"""
    print("Press c to capture the frame...")

    #Loop to continuously get frames from the webcam until user types a 'c'
    while(True):
        #Read a frame from the webcam
        ret, frame = cap.read()

        #If frame was read correctly, show it
        if ret:
            cv2.imshow("Webcam Feed", frame)

     #Break the loop when 'c' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('c'):
            break

    # Capture a frame
    ret, frame = cap.read()

    # Save the captured image
    if ret:
        cv2.imwrite(img_file, frame) # This overwrites the existing jpeg file defined in the static global variable defined above
        print("\nWebcam image captured...\n")
    else:
        print("\nError capturing image\n")

    #Release the capture object and destroy all windows
    cap.release()
    cv2.destroyAllWindows()



def open_image_in_new_window(image_path):
    """Opens an image file in a new window using Pillow."""
    try:
        img = Image.open(image_path)
        img.show()
    except FileNotFoundError:
        print(f"Error: Image file not found at '{image_path}'")
    except Exception as e:
        print(f"An error occurred: {e}")


Now we'll use these functions to open up the webcam and capture an image.  Note that the camera index might change based on your system.  Also, the 'img_file' variable value defined above has to already exist - if you get errors when trying to write the image it's likely a path issue (check what your current working directory is when running this notebook/cell) 

In [8]:
# Initialize the webcam
cap = cv2.VideoCapture(0)  # Note we need to use the correct index of the usb attached camera 

# Check if the camera opened successfully
if not cap.isOpened():
    raise IOError("Cannot open webcam - make sure it is attached")

# Create a window named "Webcam Feed" to display what the webcam is seeing on to the console
cv2.namedWindow("Webcam Feed")

# Call the capture frame function here
get_image_frame_from_webcam()

# Call the function to open the webcam image in a new window and show it on the console
open_image_in_new_window(img_file)

Press c to capture the frame...

Webcam image captured...



Now we'll upload the image file into an Azure storage blob so that we can use it as input (via it's public url) into various Azure Vision API calls

In [ ]:
# Now write the image into the azure blob storage container so that we can run various Azure AI tools against it...
container_name = "ai-vision-test"  # This is the storage container created in the Azure environment
blob_name = "sort-object"  # The name to assign to the uploaded file

# Upload the file and print the URL to the screen
uploaded_file_url = upload_blob(img_file, container_name, blob_name)
print("\nUploaded file URL in Azure is: ", uploaded_file_url)